<a href="https://colab.research.google.com/github/anmorgan24/COVID-Weather-Capstone/blob/main/basic_works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install comet_ml torch torchvision tqdm --quiet

     |████████████████████████████████| 441 kB 13.4 MB/s 
     |████████████████████████████████| 502 kB 70.9 MB/s 
     |████████████████████████████████| 54 kB 895 kB/s 
     |████████████████████████████████| 54 kB 1.6 MB/s 
     |████████████████████████████████| 168 kB 63.2 MB/s 
     |████████████████████████████████| 139 kB 11.0 MB/s 
     |████████████████████████████████| 140 kB 62.6 MB/s 
     |████████████████████████████████| 127 kB 29.1 MB/s 
     |████████████████████████████████| 168 kB 63.0 MB/s 
     |████████████████████████████████| 166 kB 23.2 MB/s 
     |████████████████████████████████| 166 kB 38.7 MB/s 
     |████████████████████████████████| 162 kB 27.3 MB/s 
     |████████████████████████████████| 162 kB 17.7 MB/s 
     |████████████████████████████████| 158 kB 67.4 MB/s 
     |████████████████████████████████| 157 kB 77.9 MB/s 
     |████████████████████████████████| 157 kB 63.2 MB/s 
     |████████████████████████████████| 157 kB 14.2 MB/s 
     |████████████

In [2]:
import comet_ml
from comet_ml import Experiment 

experiment = Experiment(
    api_key = "3VvcYhC4R9zIJ989bXfPGqx7P",
    project_name = "datapanels",
    workspace = "anmorgan24"
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET ERROR: Failed to calculate active processors count. Fall back to default CPU count 1
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. You can override where Comet is looking for a Git Patch by setting the configuration `COMET_GIT_DIRECTORY`
COMET INFO: Experiment is live on comet.com https://www.comet.com/anmorgan24/datapanels/0205ec70a4d8431c85c0f32a7469c63f



In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.autograd import Variable
from tqdm import tqdm
import pandas as pd
import numpy as np

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d mlg-ulb/creditcardfraud  
!unzip creditcardfraud.zip

 99% 65.0M/66.0M [00:02<00:00, 42.3MB/s]
100% 66.0M/66.0M [00:02<00:00, 31.2MB/s]
Archive:  creditcardfraud.zip
  inflating: creditcard.csv          


In [5]:
data = pd.read_csv('creditcard.csv')

In [6]:
data = data.drop(['Time'],axis = 1)

In [12]:
from sklearn.model_selection import train_test_split

In [15]:
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42, stratify = data["Class"])

In [16]:
hyper_params = {
    "learning_rate": 1e-1/10,
    "epochs": 2,
    "batch_size": 32,
    "weight_decay": 1e-8
}
experiment.log_parameters(hyper_params)

In [17]:
import torch.utils.data as data_utils

In [18]:
train_target = torch.tensor(train_data['Class'].values.astype(np.float32))
train = torch.tensor(train_data.drop('Class', axis = 1).values.astype(np.float32)) 
train_tensor = data_utils.TensorDataset(train, train_target) 
train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = 100, shuffle = True)

In [19]:
val_target = torch.tensor(val_data['Class'].values.astype(np.float32))
val = torch.tensor(val_data.drop('Class', axis = 1).values.astype(np.float32)) 
val_tensor = data_utils.TensorDataset(val, val_target) 
val_loader = data_utils.DataLoader(dataset = val_tensor, batch_size = 100, shuffle = True)

In [20]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(29,14),
            nn.Tanh(),
            nn.Linear(14,7),
            nn.LeakyReLU(),
            )
        self.decoder = nn.Sequential(
            nn.Linear(7, 7),
            nn.Tanh(),
            nn.Linear(7, 29),
            nn.LeakyReLU()
            )
    def forward(self , x):
        x = self.encoder(x)
        x = self.decoder(x)
        return(x)

In [21]:
model = Autoencoder()

In [22]:
# Loss and Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=hyper_params["learning_rate"], weight_decay=10e-05)

In [23]:
num_epochs = 2

In [26]:
history = {}
history['train_loss'] = []
history['test_loss'] = []

In [29]:
df = pd.DataFrame()

In [30]:
for epoch in range(num_epochs):
    h = np.array([])
    for data, label in train_loader:
        #print(type(data))
        #data = Variable(data).cpu()
        #print(type(data))
        # ===================forward=====================
        output = model(data)
        loss = criterion(output, data)
        h = np.append(h, loss.item())
        
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    mean_loss = np.mean(h)
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, mean_loss))
    history['train_loss'].append(mean_loss)


epoch [1/2], loss:1363.3688
epoch [2/2], loss:1483.6301
logging table


NameError: ignored

In [ ]:

    if epoch == (hyper_params["epochs"] - 1):
        print("logging table")
        df = pd.DataFrame()
        df['loss'] = losses
        df['label'] = labels
        experiment.log_table('test.csv', df)
        #experiment.log_table("vectors.csv", [["one", "two", "three"],[1, 2, 3],[4, 5, 6]])
    

#torch.save(model.state_dict(), './credit_card_model.pth')

In [28]:
experiment.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/anmorgan24/datapanels/0205ec70a4d8431c85c0f32a7469c63f
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [684] : (1.7330440282821655, 120725.8125)
COMET INFO:   Parameters:
COMET INFO:     batch_size    : 32
COMET INFO:     epochs        : 2
COMET INFO:     learning_rate : 0.01
COMET INFO:     weight_decay  : 1e-08
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     model graph         : 1
COMET INFO:     notebook            : 2
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: ---------------------------
COMET INFO: Uploading metrics, params, and assets to Comet before program terminati

In [4]:
hyper_params = {"batch_size": 100,
                "num_epochs": 2,
                "learning_rate": 0.001}

experiment.log_parameters(hyper_params)

In [5]:
#!rmdir ~/.kaggle

rmdir: failed to remove '/root/.kaggle': No such file or directory


In [19]:
transactions.Amount = MinMaxScaler().fit_transform(
transactions.Amount.values.reshape(-1,1)
)


In [20]:
transactions.head(5)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.005824,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0.000105,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0.014739,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.004807,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0.002724,0


In [28]:
y = transactions["Class"]
X = transactions.drop("Class", axis = 1)

In [30]:
X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)

In [37]:
X_train_tensor = torch.tensor(X_train)

<ipython-input-37-87deff5b11b7>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensor = torch.tensor(X_train)


In [43]:
# create tensor datasets from df
X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)
y_train = torch.tensor(y_train.values)
y_test = torch.tensor(y_test.values)
train_ds = TensorDataset(X_train, y_train)

<ipython-input-43-39f142faf38d>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
<ipython-input-43-39f142faf38d>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)


RuntimeError: ignored

In [21]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(29,14),
            nn.Tanh(),
            nn.Linear(14,7),
            nn.LeakyReLU(),
            )
        self.decoder = nn.Sequential(
            nn.Linear(7, 7),
            nn.Tanh(),
            nn.Linear(7, 29),
            nn.LeakyReLU()
            )
    def forward(self , x):
        x = self.encoder(x)
        x = self.decoder(x)
        return(x)

In [17]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:
num_epochs = 100
minibach_size = 32
lr = 1e-3


In [23]:
import torch.utils.data as data_utils

In [24]:
train_loader = data_utils.DataLoader(X_train,batch_size=minibach_size,shuffle=True)
test_loader = data_utils.DataLoader(X_test, batch_size=1, shuffle=False)

In [25]:

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
model.parameters(), lr=lr, weight_decay=10e-05)


In [26]:
history = {}
history['train_loss'] = []
history['test_loss'] = []

In [29]:
model = Autoencoder()
#.to(device)

In [30]:
for epoch in range(num_epochs):
    h = np.array([])
    for data in train_loader:
        #print(type(data))
        #data = Variable(data).cpu()
        #print(type(data))
        # ===================forward=====================
        output = model(data)
        loss = criterion(output, data)
        h = np.append(h, loss.item())
        
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    mean_loss = np.mean(h)
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, mean_loss))
    history['train_loss'].append(mean_loss)
    

#torch.save(model.state_dict(), './credit_card_model.pth')

RuntimeError: ignored

In [7]:
pc_fraud = len(data.loc[data['Class'] == 1].values)/len(data.loc[data['Class'] == 0].values)
print(pc_fraud*100)
print(data.head())

0.17304750013189596
   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26 

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from collections import Counter


In [9]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [9]:
data.shape

(284807, 31)

In [10]:
data.iloc[:2000, :].shape

(2000, 31)

In [11]:
data = data.iloc[:2000,:]

In [12]:

# Original dataset
x = data.drop('Class', axis=1).values
y = data['Class'].values
scaler = MinMaxScaler()
scaler.fit(x)
x = scaler.transform(x)
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=0)

print('Sampled train dataset shape %s' % Counter(ytrain))
print('Sampled validation dataset shape %s' % Counter(ytest))

Sampled train dataset shape Counter({0: 1598, 1: 2})
Sampled validation dataset shape Counter({0: 400})


In [28]:
'''x = data.drop(["Class"], axis=1)
y = data["Class"].values

x_scale = MinMaxScaler().fit_transform(x.values)
x_norm, x_fraud = x_scale[y == 0], x_scale[y == 1]'''

In [25]:
bs = 100
#creating torch dataset and loader using original dataset. 
#to use resampled dataset, replace ex. xtrain with xtrain_over etc.

train_ds = torch.utils.data.TensorDataset(torch.tensor(xtrain).float(), torch.tensor(ytrain).float())
valid_ds = torch.utils.data.TensorDataset(torch.tensor(xtest).float(), torch.tensor(ytest).float())

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=bs)
valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=bs)

TypeError: ignored

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(29,14),
            nn.Tanh(),
            nn.Linear(14,7),
            nn.LeakyReLU(),
            )
        self.decoder = nn.Sequential(
            nn.Linear(7, 7),
            nn.Tanh(),
            nn.Linear(7, 29),
            nn.LeakyReLU()
            )
    def forward(self , x):
        x = self.encoder(x)
        x = self.decoder(x)
        return(x)

model = Autoencoder().to(device)

# Loss and Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=hyper_params["learning_rate"], weight_decay=10e-05)

In [20]:
history = {}
history['train_loss'] = []
history['test_loss'] = []

In [21]:
num_epochs = 2
minibach_size = 32

In [23]:
for epoch in range(num_epochs):
    h = np.array([])
    for data in train_dl:
        #print(type(data))
        #data = Variable(data).cpu()
        #print(type(data))
        # ===================forward=====================
        output = model(data)
        loss = criterion(output, data)
        h = np.append(h, loss.item())
        
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    mean_loss = np.mean(h)
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, mean_loss))
    history['train_loss'].append(mean_loss)
    

#torch.save(model.state_dict(), './credit_card_model.pth')

TypeError: ignored

In [21]:
def train(model, optimizer, criterion, dataloader, epoch, experiment):
    model.train()
    total_loss = 0
    correct = 0
    for e in num_epochs:
        tqdm(dataloader, total=len(dataloader))):
        optimizer.zero_grad()
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)

        loss = criterion(outputs, labels)
        pred = outputs.argmax(
            dim=1, keepdim=True
        )  # get the index of the max log-probability

        loss.backward()
        optimizer.step()

        # Compute train accuracy
        batch_correct = pred.eq(labels.view_as(pred)).sum().item()
        batch_total = labels.size(0)

        total_loss += loss.item()
        correct += batch_correct

        # Log batch_accuracy to Comet; step is each batch
        experiment.log_metric("batch_accuracy", batch_correct / batch_total)

    total_loss /= len(dataloader.dataset)
    correct /= len(dataloader.dataset)

    experiment.log_metrics({"accuracy": correct, "loss": total_loss}, epoch=epoch)

IndentationError: ignored

In [19]:
for epoch in range(num_epochs):
    h = np.array([])
    for data in train_dl:
        #print(type(data))
        #data = Variable(data).cpu()
        #print(type(data))
        # ===================forward=====================
        output = model(data)
        loss = criterion(output, data)
        h = np.append(h, loss.item())
        
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    mean_loss = np.mean(h)
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, mean_loss))
    history['train_loss'].append(mean_loss)

TypeError: ignored

In [20]:
class Classifier(torch.nn.Module):
    def __init__(self, n_input=10, n_hidden = 20, n_output = 1,drop_prob=0.5):
        super().__init__()
        self.extractor1 = torch.nn.Linear(n_input, n_hidden)
        self.extractor2 = torch.nn.Linear(n_hidden, n_hidden)
        self.relu = torch.nn.ReLU()
        self.drop_out = torch.nn.Dropout(drop_prob)
        self.classifier = torch.nn.Linear(n_hidden, n_output)

    def forward(self, xb):
        x = self.relu(self.extractor1(xb))
        x = self.relu(self.extractor2(x))
        x = self.drop_out(x)
        return self.classifier(x).squeeze()

In [18]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

In [21]:
model = Classifier().to(device)
# Loss and Optimizer
#criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=hyper_params["learning_rate"], momentum=0.9)

In [22]:
#training the network
def train(epochs, model, loss_func, opt, train_dl, valid_dl):
    total_loss = 0
    correct = 0
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)

        # Compute train accuracy
        batch_correct = pred.eq(labels.view_as(pred)).sum().item()
        batch_total = labels.size(0)

        total_loss += loss.item()
        correct += batch_correct

        # Log batch_accuracy to Comet; step is each batch
        experiment.log_metric("batch_accuracy", batch_correct / batch_total)

        total_loss /= len(dataloader.dataset)
        correct /= len(dataloader.dataset)

        experiment.log_metrics({"accuracy": correct, "loss": total_loss}, epoch=epoch)
        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        total_loss /= len(dataloader.dataset)
        correct /= len(dataloader.dataset)

        experiment.log_metrics({"accuracy": correct, "loss": total_loss}, epoch=epoch)

In [ ]:
df = pd.DataFrame()

for epoch in range(hyper_params["epochs"]):
    count = 0
    losses = []
    for (image, _) in train_loader:
       
      # Reshaping the image to (-1, 784)
        image = image

      # Output of Autoencoder
        reconstructed = model(image)

        batch_size = reconstructed.shape[0]

      # Calculating the loss function
        loss = loss_function(reconstructed, image)
       
      # The gradients are set to zero,
      # the gradient is computed and stored.
      # .step() performs parameter update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.detach().cpu().numpy())
    experiment.log_metric('epoch_loss', np.mean(losses), epoch=epoch+1)

    count = 0
    counts = []
    losses = []
    labels = []
    for (image, label) in val_loader:
        
      image = image
      # Output of Autoencoder
      reconstructed = model(image)
      loss = loss_function(reconstructed, image)
      experiment.log_image(image.reshape(28,28), name="sample_%d_og.png" % count)
      experiment.log_image(reconstructed.reshape(28,28), name="sample_%d_recon.png" % count)
      counts.append(count)
      losses.append(loss.detach().cpu().numpy())
      labels.append(label[0].detach().cpu().numpy())
      
      count += 1

    if epoch == (hyper_params["epochs"] - 1):
        print("logging table")
        df = pd.DataFrame()
        df['loss'] = losses
        df['label'] = labels
        experiment.log_table('test.csv', df)

In [23]:
def test(model, optimizer, criterion, dataloader, epoch, experiment):
    model.eval()

    total_loss = 0
    correct = 0
    with torch.no_grad():
        for batch_idx, (images, labels) in tqdm(enumerate(dataloader)):
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            total_loss += criterion(output, labels).item()  # sum up batch loss
            pred = output.argmax(
                dim=1,
                keepdim=True,
            )  # get the index of the max log-probability
            correct += pred.eq(labels.view_as(pred)).sum().item()

        total_loss /= len(dataloader.dataset)
        correct /= len(dataloader.dataset)

        experiment.log_metrics({"accuracy": correct, "loss": total_loss}, epoch=epoch)

In [25]:
#network setting
n_input = xtrain.shape[1]
n_output = 1
n_hidden = 15

model = Classifier(n_input=n_input,n_hidden=n_hidden,n_output=n_output,drop_prob=0.2)

lr = 0.001

#for orignal dataset, I use pos_weight.
pos_weight = torch.tensor([5])
opt = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
loss_func = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)

n_epoch = 2

In [26]:
train(n_epoch,model,loss_func,opt,train_dl,valid_dl)
model.eval()

NameError: ignored

In [2]:
import os
from comet_ml import Experiment
import torch
from torchvision import datasets
from torchvision import transforms
import numpy as np
import torch.nn.functional as F
import pandas as pd


In [3]:

experiment = Experiment(
    api_key="3VvcYhC4R9zIJ989bXfPGqx7P",
    project_name="datapanels",
    workspace="anmorgan24",
)


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET ERROR: Failed to calculate active processors count. Fall back to default CPU count 1
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. You can override where Comet is looking for a Git Patch by setting the configuration `COMET_GIT_DIRECTORY`
COMET INFO: Experiment is live on comet.com https://www.comet.com/anmorgan24/datapanels/00099c496ac8441791787bfe96ca9d70



In [4]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
!kaggle datasets download -d mlg-ulb/creditcardfraud

 79% 52.0M/66.0M [00:00<00:00, 185MB/s]
100% 66.0M/66.0M [00:00<00:00, 186MB/s]


In [12]:
!unzip creditcardfraud.zip -d ~/creditcard.csv

Archive:  creditcardfraud.zip
  inflating: /root/creditcard.csv/creditcard.csv  


In [15]:
data = pd.read_csv('creditcard.csv')

In [16]:
pc_fraud = len(data.loc[data['Class'] == 1].values)/len(data.loc[data['Class'] == 0].values)
print(pc_fraud*100)
print(data.head())

0.17304750013189596
   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26 

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from collections import Counter

# Original dataset
x = data.drop('Class', axis=1).values
y = data['Class'].values
scaler = MinMaxScaler()
scaler.fit(x)
x = scaler.transform(x)
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=0)

print('Sampled train dataset shape %s' % Counter(ytrain))
print('Sampled validation dataset shape %s' % Counter(ytest))

Sampled train dataset shape Counter({0: 227454, 1: 391})
Sampled validation dataset shape Counter({0: 56861, 1: 101})


In [18]:
import torch

bs =100

#creating torch dataset and loader using original dataset. 
#to use resampled dataset, replace ex. xtrain with xtrain_over etc.
train_ds = torch.utils.data.TensorDataset(torch.tensor(xtrain).float(), torch.tensor(ytrain).float())
valid_ds = torch.utils.data.TensorDataset(torch.tensor(xtest).float(), torch.tensor(ytest).float())

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=bs)
valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=bs)

In [19]:
#network class 2-hidden layer model
class Classifier(torch.nn.Module):
    def __init__(self, n_input=10, n_hidden = 20, n_output = 1,drop_prob=0.5):
        super().__init__()
        self.extractor1 = torch.nn.Linear(n_input, n_hidden)
        self.extractor2 = torch.nn.Linear(n_hidden, n_hidden)
        self.relu = torch.nn.ReLU()
        self.drop_out = torch.nn.Dropout(drop_prob)
        self.classifier = torch.nn.Linear(n_hidden, n_output)

    def forward(self, xb):
        x = self.relu(self.extractor1(xb))
        x = self.relu(self.extractor2(x))
        x = self.drop_out(x)
        return self.classifier(x).squeeze()

In [20]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

In [21]:
#training the network
def train(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print(epoch, val_loss)


In [22]:
#network setting
n_input = xtrain.shape[1]
n_output = 1
n_hidden = 15

model = Classifier(n_input=n_input,n_hidden=n_hidden,n_output=n_output,drop_prob=0.2)

lr = 0.001

#for orignal dataset, I use pos_weight.
pos_weight = torch.tensor([5])
opt = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
loss_func = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)

n_epoch = 3

In [1]:
train(n_epoch,model,loss_func,opt,train_dl,valid_dl)
model.eval()


NameError: ignored

In [ ]:
from sklearn import metrics
ypred = model(torch.tensor(xtest).float()).detach().numpy()

ypred [ypred>=0.5] =1.0
ypred [ypred<0.5] =0.0
print('Confusion matrix: {}'. format(metrics.confusion_matrix(ytest, ypred)))
print('AUPRC score: {}'. format(metrics.average_precision_score(ytest, ypred)))
print('AUROC score: {}'.format(metrics.roc_auc_score(ytest, ypred)))
print('Accuracy score: {}'.format(metrics.accuracy_score(ytest, ypred)))
print(metrics.classification_report(ytest, ypred))

In [ ]:
hyper_params = {
    "learning_rate": 1e-1/10,
    "epochs": 150,
    "batch_size": 32,
    "weight_decay": 1e-8
}
experiment.log_parameters(hyper_params)

class FC_5_Layer(torch.nn.Module):
    def __init__(self):
        super().__init__()
         
        # Building an linear encoder with Linear
        # layer followed by Relu activation function
        # 784 ==> 9
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(28 * 28, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 36),
            torch.nn.ReLU(),
            torch.nn.Linear(36, 18),
            torch.nn.ReLU(),
            torch.nn.Linear(18, 9)
        )
         
        # Building an linear decoder with Linear
        # layer followed by Relu activation function
        # The Sigmoid activation function
        # outputs the value between 0 and 1
        # 9 ==> 784
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(9, 18),
            torch.nn.ReLU(),
            torch.nn.Linear(18, 36),
            torch.nn.ReLU(),
            torch.nn.Linear(36, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 28 * 28),
            torch.nn.Sigmoid()
        )
 
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

class FC_4_Layer(torch.nn.Module):

    def __init__(self):
        super().__init__()
         
        # Building an linear encoder with Linear
        # layer followed by Relu activation function
        # 784 ==> 9
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(28 * 28, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 36),
            torch.nn.ReLU(),
            torch.nn.Linear(36, 9)
        )
         
        # Building an linear decoder with Linear
        # layer followed by Relu activation function
        # The Sigmoid activation function
        # outputs the value between 0 and 1
        # 9 ==> 784
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(9, 36),
            torch.nn.ReLU(),
            torch.nn.Linear(36, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 28 * 28),
            torch.nn.Sigmoid()
        )
 
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

class Conv_AE(torch.nn.Module):
    def __init__(self):
        super(Conv_AE, self).__init__()
        ## encoder layers ##
        # conv layer (depth from 1 --> 16), 3x3 kernels
        self.conv1 = torch.nn.Conv2d(1, 16, 3, padding=1)  
        # conv layer (depth from 16 --> 4), 3x3 kernels
        self.conv2 = torch.nn.Conv2d(16, 4, 3, padding=1)
        # pooling layer to reduce x-y dims by two; kernel and stride of 2
        self.pool = torch.nn.MaxPool2d(2, 2)
        
        ## decoder layers ##
        ## a kernel of 2 and a stride of 2 will increase the spatial dims by 2
        self.t_conv1 = torch.nn.ConvTranspose2d(4, 16, 2, stride=2)
        self.t_conv2 = torch.nn.ConvTranspose2d(16, 1, 2, stride=2)


    def forward(self, x):
        ## encode ##
        # add hidden layers with relu activation function
        # and maxpooling after
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        # add second hidden layer
        x = F.relu(self.conv2(x))
        x = self.pool(x)  # compressed representation
        
        ## decode ##
        # add transpose conv layers, with relu activation function
        x = F.relu(self.t_conv1(x))
        # output layer (with sigmoid for scaling from 0 to 1)
        x = torch.sigmoid(self.t_conv2(x))
                
        return x




# Transforms images to a PyTorch Tensor
tensor_transform = transforms.ToTensor()
 
# Download the MNIST Dataset
dataset = datasets.MNIST(root = "./data",
                         train = True,
                         download = True,
                         transform = tensor_transform)
 

train_dataset = torch.utils.data.Subset(dataset,list(range(0, 1000)))
val_dataset = torch.utils.data.Subset(dataset,list(range(1000, 1010)))

# DataLoader is used to load the dataset
# for training
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                     batch_size = hyper_params["batch_size"])
                        
val_loader = torch.utils.data.DataLoader(dataset = val_dataset,
                                     batch_size = 1)

# Model Initialization
model = Conv_AE()
# Validation using MSE Loss function
loss_function = torch.nn.MSELoss()
 
# Using an Adam Optimizer with lr = 0.1
optimizer = torch.optim.Adam(model.parameters(),
                             lr = hyper_params["learning_rate"],
                             weight_decay = hyper_params["weight_decay"])


df = pd.DataFrame()

for epoch in range(hyper_params["epochs"]):
    count = 0
    losses = []
    for (image, _) in train_loader:
       
      # Reshaping the image to (-1, 784)
        image = image

      # Output of Autoencoder
        reconstructed = model(image)

        batch_size = reconstructed.shape[0]

      # Calculating the loss function
        loss = loss_function(reconstructed, image)
       
      # The gradients are set to zero,
      # the gradient is computed and stored.
      # .step() performs parameter update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.detach().cpu().numpy())
    experiment.log_metric('epoch_loss', np.mean(losses), epoch=epoch+1)

    count = 0
    counts = []
    losses = []
    labels = []
    for (image, label) in val_loader:
        
      image = image
      # Output of Autoencoder
      reconstructed = model(image)
      loss = loss_function(reconstructed, image)
      experiment.log_image(image.reshape(28,28), name="sample_%d_og.png" % count)
      experiment.log_image(reconstructed.reshape(28,28), name="sample_%d_recon.png" % count)
      counts.append(count)
      losses.append(loss.detach().cpu().numpy())
      labels.append(label[0].detach().cpu().numpy())
      
      count += 1

    if epoch == (hyper_params["epochs"] - 1):
        print("logging table")
        df = pd.DataFrame()
        df['loss'] = losses
        df['label'] = labels
        experiment.log_table('test.csv', df)
        experiment.log_table("vectors.csv", [["one", "two", "three"],[1, 2, 3],[4, 5, 6]])

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

logging table


In [ ]:
experiment.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/anmorgan24/datapanels/831e17a7c9bc44639b05347e8364c44b
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     epoch_loss [150] : (0.02031000517308712, 0.14468920230865479)
COMET INFO:     loss [480]       : (0.01898754946887493, 0.2544822692871094)
COMET INFO:   Parameters:
COMET INFO:     batch_size    : 32
COMET INFO:     epochs        : 150
COMET INFO:     learning_rate : 0.01
COMET INFO:     weight_decay  : 1e-08
COMET INFO:   Uploads:
COMET INFO:     asset               : 1 (26 bytes)
COMET INFO:     dataframe           : 1 (173 bytes)
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     images              : 3000
COMET INFO:     installed packages  : 1
COMET INFO:     model graph         : 1
COM